In [ ]:
This module allows you to test RNet on images and videos.

In [ ]:
# Install required libraries
!pip install torch
!pip install torchvision
!pip install numpy
!pip install tqdm
!pip install albumentations==0.4.6
!pip install natsort
!pip install opencv-python
import platform
if platform.python_version() >= "3.10":
    import glob
else:
    %pip install glob2
    import glob2 as glob

In [ ]:
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch
import torchvision
from torchvision import transforms
from PIL import Image
from utils.model import RNet

In [ ]:
# ONLY USE IT IF YOU WANT TO TEST RNET ON VIDEOS
# Test the model on video, using saved model
counter = 0
WIDTH, HEIGHT = 1280, 720
IMAGE_HEIGHT = 80  # Do not change this variable
IMAGE_WIDTH = 120  # Do not change this variable
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TEST_DIR = "PATH/TO/THE/VIDEO/YOU/WANT/TO/TEST/ON" # Change this to your actual test video

model = RNet(in_channels=3, out_channels=1).to(DEVICE)
model.load_state_dict(torch.load("model/checkpoint.pth"))
video = cv2.VideoCapture(TEST_DIR)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('videos/test-video.avi', fourcc, 30.0, (int(WIDTH), int(HEIGHT))) # The video will be at videos/test-video.avi

mean=[0.0, 0.0, 0.0]
std=[1.0, 1.0, 1.0]
TRANSFORM_IMG = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
  transforms.Normalize(mean, std)
  ])

# Code to test RNet on videos
model.eval()
with torch.no_grad():
  while True:
    ret, frame = video.read()

    # Get predictions from frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (WIDTH, HEIGHT))
    pilimg = np.array(Image.fromarray(frame).convert("RGB"))
    img = TRANSFORM_IMG(pilimg)
    img = img.to(DEVICE).unsqueeze(0)
    predictions = torch.sigmoid(model(img))
    predictions = (predictions > 0.99).float()

    # Transform Tensor to PIL Image, then convert PIL Image to cv2 image
    pil_img = transforms.ToPILImage()(predictions.squeeze(0))
    opencvImage = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    opencvImage = cv2.erode(opencvImage, np.ones((1, 2)))
    opencvImage = cv2.dilate(opencvImage, np.ones((2, 2)))
    opencvImage = ((opencvImage > 10) * 255).astype(np.uint8)
    opencvImage = cv2.GaussianBlur(opencvImage, (5, 5), 0)

    # Remove black background
    bgr = opencvImage[:,:,:3]
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    alpha = opencvImage[:,:,2]
    result = np.dstack([bgr, alpha])

    # Overlay road image and predictions image
    opencvImage = cv2.resize(result, (WIDTH, HEIGHT))
    opencvImage = cv2.addWeighted(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), 1, opencvImage[:,:,:3], 1, 0)

    # Result
    out.write(opencvImage)

In [ ]:
# ONLY USE IT IF YOU WANT TO TEST RNET ON IMAGES
# Test the model on an image, using saved model
counter = 0
WIDTH, HEIGHT = 1280, 720
IMAGE_HEIGHT = 80  # Do not change this variable
IMAGE_WIDTH = 120  # Do not change this variable
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_PATH = "PATH/TO/THE/IMAGE/YOU/WANT/TO/TEST/ON" # Change this to your actual test image

model = RNet(in_channels=3, out_channels=1).to(DEVICE)
model.load_state_dict(torch.load("model/checkpoint.pth"))

mean=[0.0, 0.0, 0.0]
std=[1.0, 1.0, 1.0]
TRANSFORM_IMG = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
  transforms.Normalize(mean, std)
  ])

model.eval()
with torch.no_grad():
  pilimg = np.array(Image.open(IMAGE_PATH).convert("RGB"))
  img = TRANSFORM_IMG(pilimg)
  img = img.to(DEVICE).unsqueeze(0)
  predictions = torch.sigmoid(model(img))
  predictions = (predictions > 0.99).float()
  preds_to_pil_img = transforms.ToPILImage()(predictions.squeeze(0))
  cv2_img = cv2.cvtColor(np.array(preds_to_pil_img), cv2.COLOR_BGR2RGB)
  cv2_img = cv2.erode(cv2_img, np.ones((1, 2)))
  cv2_img = cv2.dilate(cv2_img, np.ones((2, 2)))
  cv2_img = ((cv2_img > 10) * 255).astype(np.uint8)
  cv2_img = cv2.GaussianBlur(cv2_img, (5, 5), 0)

  # Remove black background
  bgr = cv2_img[:,:,:3]
  gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
  bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
  alpha = cv2_img[:,:,2]
  result = np.dstack([bgr, alpha])

  # Overlay road image and predictions image
  cv2_img = cv2.resize(result, (WIDTH, HEIGHT))
  cv2_img = cv2.addWeighted(cv2.cvtColor(pilimg, cv2.COLOR_RGB2BGR), 1, cv2_img[:,:,:3], 1, 0)

  cv2.imwrite("images/predicted_image.jpg", cv2_img)